In [ ]:
#Load Packages and Functions
import glob
import nltk
import numpy as np
import os
import pandas as pd
import pickle
import scipy,stanza,sys,re,pickle
import matplotlib.pyplot as plt

from collections import Counter
from fuzzywuzzy import fuzz
from gensim.models import Word2Vec
from iocparser import IOCParser
import networkx as nx
from nltk.corpus import words
from scipy import spatial
from simpletransformers.ner import NERModel
from stanza.server import CoreNLPClient
import stanza.protobuf.CoreNLP_pb2 as CoreNLP_pb2
from tqdm.contrib import tzip

import stanza
corenlp_dir = '/home/user/corenlp'
from stanza.server import CoreNLPClient
import os
os.environ["CORENLP_HOME"] ='/home/user/corenlp'
word_set = set(words.words())
sys.modules['CoreNLP_pb2'] = CoreNLP_pb2

w2v = pickle.load(open('pkl/w2v_ref_blog_lemma.pkl', 'rb'))

mitre_groups = pickle.load(open('pkl/mitre_groups.pkl', 'rb'))
mitre_malwares = pickle.load(open('pkl/mitre_malwares.pkl', 'rb'))
programm_language = pickle.load(open('pkl/programming_language.pkl', 'rb'))

mitre_groups = set(list(map(str.lower, mitre_groups)))
mitre_malwares = set(list(map(str.lower, mitre_malwares)))

malware = pickle.load(open('pkl/malware.pkl', 'rb'))
software = pickle.load(open('pkl/software.pkl', 'rb'))
actor = pickle.load(open('pkl/actor.pkl', 'rb'))

mal_suffix = ['rat', 'downloader', 'trojan', 'ransomware', 'malware', 'virus', 'worm', 'backdoor']
for malname in list(malware):
    if ' ' in malname:
        for _ in mal_suffix:
            if _ in malname.split(' '):
                malware.remove(malname)
                malware.add(''.join(filter(lambda x: x not in mal_suffix, malname.split(' '))))

for malname in list(malware):
    if malname in word_set:
        malware.remove(malname)

for softname in list(software):
    if softname in word_set:
        software.remove(softname)

for actname in list(actor):
    if actname in word_set:
        actor.remove(actname)

#%%
import stanza
corenlp_dir = '/home/user/corenlp'
from stanza.server import CoreNLPClient
import os
os.environ["CORENLP_HOME"] ='/home/user/corenlp'
word_set = set(words.words())
sys.modules['CoreNLP_pb2'] = CoreNLP_pb2


w2v = pickle.load(open('pkl/w2v_ref_blog_lemma.pkl', 'rb'))
mitre_groups = pickle.load(open('pkl/mitre_groups.pkl', 'rb'))
mitre_malwares = pickle.load(open('pkl/mitre_malwares.pkl', 'rb'))
programm_language = pickle.load(open('pkl/programming_language.pkl', 'rb'))

mitre_groups = set(list(map(str.lower, mitre_groups)))
mitre_malwares = set(list(map(str.lower, mitre_malwares)))

malware = pickle.load(open('pkl/malware.pkl', 'rb'))
software = pickle.load(open('pkl/software.pkl', 'rb'))
actor = pickle.load(open('pkl/actor.pkl', 'rb'))

mal_suffix = ['rat', 'downloader', 'trojan', 'ransomware', 'malware', 'virus', 'worm', 'backdoor']
for malname in list(malware):
    if ' ' in malname:
        for _ in mal_suffix:
            if _ in malname.split(' '):
                malware.remove(malname)
                malware.add(''.join(filter(lambda x: x not in mal_suffix, malname.split(' '))))

for malname in list(malware):
    if malname in word_set:
        malware.remove(malname)

for softname in list(software):
    if softname in word_set:
        software.remove(softname)

for actname in list(actor):
    if actname in word_set:
        actor.remove(actname)

class Triple():
    def __init__(self, sub, rel, obj, sub_ent='', rel_ent='', obj_ent='', sub_ent_conf=0, obj_ent_conf=0, conf=0):
        self.sub = sub.lower()
        self.rel = rel.lower()
        self.obj = obj.lower()
        self.sub_ent = sub_ent
        self.rel_ent = rel_ent
        self.obj_ent = obj_ent
        self.sub_ent_conf = sub_ent_conf
        self.obj_ent_conf = obj_ent_conf
        self.confidence = conf

    def __hash__(self):
        return hash(self.simple_sub + self.simple_rel + self.simple_obj)

    def set_raw_article_doc(self, doc):
        self.article_doc = doc

    def set_raw_sent(self, sent):
        self.raw_sent = sent

    def set_raw_triple(self, triple_obj):
        self.triple_obj = triple_obj

    def set_sent_ind(self, sent_ind):
        self.sent_ind = sent_ind

    def set_article_ref(self, article):
        self.article_id = article

    def set_tactic(self, tactic):
        self.tactic = tactic

    def set_tactic_conf(self, tactic_conf):
        self.tactic_conf = tactic_conf

    def set_behave_conf(self, behave_conf):
        self.behave_conf = behave_conf

    def set_relative_sent_ind(self, relative_sent_order):
        self.relative_sent_order = relative_sent_order

    def set_count(self, count):
        self.count = count

    def add_cluster(self, cluster):
        self.from_cluster = set()
        self.from_cluster.add(cluster)

    def __eq__(self, other):
        sub_eq = False
        if self.sub in other.sub or other.sub in self.sub:
            sub_eq = True
        elif fuzz.ratio(self.sub, other.sub) > 90:
            sub_eq = True
        else:
            self_sub_tokens = self.sub.split(' ')
            other_sub_tokens = other.sub.split(' ')
            if all(map(lambda x: x in w2v.wv, self_sub_tokens)) and all(map(lambda x: x in w2v.wv, other_sub_tokens)):
                self_sub_vec = np.mean([w2v.wv[x] for x in self_sub_tokens], axis=0)
                other_sub_vec = np.mean([w2v.wv[x] for x in other_sub_tokens], axis=0)
                sub_eq = spatial.distance.cosine(self_sub_vec, other_sub_vec) < 0.3

        rel_eq = False
        if self.rel in other.rel or other.rel in self.rel:
            rel_eq = True
        elif self.rel == other.rel:
            rel_eq = True
        else:
            self_rel_tokens = self.rel.split(' ')
            other_rel_tokens = other.rel.split(' ')
            if all(map(lambda x: x in w2v.wv, self_rel_tokens)) and all(map(lambda x: x in w2v.wv, other_rel_tokens)):
                self_rel_vec = np.mean([w2v.wv[x] for x in self_rel_tokens], axis=0)
                other_rel_vec = np.mean([w2v.wv[x] for x in other_rel_tokens], axis=0)
                rel_eq = spatial.distance.cosine(self_rel_vec, other_rel_vec) < 0.3

        obj_eq = False
        if self.obj in other.obj or other.obj in self.obj:
            obj_eq = True
        elif fuzz.ratio(self.obj, other.obj) > 90:
            obj_eq = True
        else:
            self_obj_tokens = self.obj.split(' ')
            other_obj_tokens = other.obj.split(' ')
            if all(map(lambda x: x in w2v.wv, self_obj_tokens)) and all(map(lambda x: x in w2v.wv, other_obj_tokens)):
                self_obj_vec = np.mean([w2v.wv[x] for x in self_obj_tokens], axis=0)
                other_obj_vec = np.mean([w2v.wv[x] for x in other_obj_tokens], axis=0)
                obj_eq = spatial.distance.cosine(self_obj_vec, other_obj_vec) < 0.3

        return sub_eq and rel_eq and obj_eq and self.sub_ent == other.sub_ent and self.obj_ent == other.obj_ent and self.rel_ent == other.rel_ent

    def print(self):
        use_less='1'

def bert_ner(ann):
    tokenized = [[x.value for x in sent.token] for sent in ann.sentence]
    predictions, raw_outputs = model.predict(tokenized, split_on_space=False)
    return predictions, raw_outputs

def simple_match(candidate, ioc_dic):
    candidate = candidate.lower().strip()
    if (candidate.startswith("cve")):
        return "CVE"
    for _ in mitre_groups:
        if candidate.find(_) != -1:
            return "Known_Actor"
    for _ in mitre_malwares:
        if candidate.find(_) != -1:
            return "Known_Malware"
    for _ in programm_language:
        if _ == candidate:
            return "Programming_Language"
    for _ in ioc_dic:
        if _ in candidate:
            textobj = IOCParser(ioc_dic[_]);
            results = textobj.parse();
            if len(results) > 0:
                return results[0].kind
    return ''


def process_triple_tokens(tokens, ann, predictions, raws, ioc_dic):
    redundant = ['PRP$', 'JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'DT']
    inflex = ['VBD', 'VBG', 'VBP', 'VBZ', 'NNS', 'NNPS']
    sub_tokens = tokens
    simplified_sub_tokens = []
    sub_entity = ''
    sub_ner_conf = 0
    for sub_token in sub_tokens:
        token_obj = ann.sentence[sub_token.sentenceIndex].token[sub_token.tokenIndex]
        if token_obj.pos.startswith('NN'):
            ner_tag = 'O'
            try:
                ner_tag = list(predictions[sub_token.sentenceIndex][sub_token.tokenIndex].values())[0]
                sub_ner_conf = max(
                    scipy.special.softmax(list(raws[sub_token.sentenceIndex][sub_token.tokenIndex].values())[0][0]))
            except:
                pass
            if ner_tag.startswith('B'):
                sub_entity = ner_tag.split('-')[1]
            if token_obj.pos in inflex:
                simplified_sub_tokens.append(token_obj.lemma)
            else:
                simplified_sub_tokens.append(token_obj.word)
        elif token_obj.pos not in redundant:
            simplified_sub_tokens.append(token_obj.word)
    if len(simplified_sub_tokens) == 0:
        return None, None, None
    else:
        sub = " ".join(simplified_sub_tokens)
        simple_ent = simple_match(sub, ioc_dic)
        if simple_ent != '':
            sub_entity = simple_ent
            sub_ner_conf = 1
    return sub, sub_entity, sub_ner_conf


def extract_triples_with_ner(ann, predictions, raws, ioc_dic, doc_id=None):
    inflex = ['VBD', 'VBG', 'VBP', 'VBZ', 'NNS', 'NNPS']
    extracted = list()
    sub_counter = Counter()
    obj_counter = Counter()
    relation_counter = Counter()
    for sent_ind, sent in enumerate(ann.sentence):
        triples = sent.openieTriple
        for triple in triples:
            #print(triple)
            sub_tokens = triple.subjectTokens
            sub, sub_entity, sub_ner_conf = process_triple_tokens(sub_tokens, ann, predictions, raws, ioc_dic)
            if not sub:
                continue

            obj_tokens = triple.objectTokens
            obj, obj_entity, obj_ner_conf = process_triple_tokens(obj_tokens, ann, predictions, raws, ioc_dic)
            if not obj:
                continue

            rel_tokens = triple.relationTokens
            simplified_rel_tokens = []
            for rel_token in rel_tokens:
                token_obj = ann.sentence[rel_token.sentenceIndex].token[rel_token.tokenIndex]
                if token_obj.pos != 'RB':
                    if token_obj.pos in inflex:
                        simplified_rel_tokens.append(token_obj.lemma)
                    else:
                        simplified_rel_tokens.append(token_obj.word)
            if len(simplified_rel_tokens) == 0:
                #print("no rel")
                continue
            else:
                rel = " ".join(simplified_rel_tokens)

            for k, v in ioc_dic.items():
                if k in sub:
                    sub = sub.replace(k, v)
                if k in obj:
                    obj = obj.replace(k, v)
                if k in rel:
                    rel = rel.replace(k, v)

            tri = Triple(sub, rel, obj, sub_ent=sub_entity, obj_ent=obj_entity, sub_ent_conf=sub_ner_conf,
                         obj_ent_conf=obj_ner_conf, conf=triple.confidence)
            tri.set_sent_ind(sent_ind)
            tri.set_relative_sent_ind(sent_ind / len(ann.sentence))
            tri.set_article_ref(doc_id)
            tri.set_raw_triple(triple)
            extracted.append(tri)
            sub_counter.update([sub])
            obj_counter.update([obj])
            relation_counter.update([rel])
    return extracted, (sub_counter, obj_counter), relation_counter


def change_ioc(text):
    count = 0
    dic = dict()
    textobj = IOCParser(text);
    results = textobj.parse();
    for res in results:
        sig = res.kind + str(count)
        dic[sig] = res.value
        count += 1
        text = text.replace(res.value, sig)
    return text, dic

tactic_dic = {
    0: 'Initial Access', 
    1: 'Execution', 
    2: 'Defense Evasion', 
    3: 'Command and Control', 
    4: 'Privilege Escalation', 
    5: 'Persistence', 
    6: 'Lateral Movement', 
    7: 'DataLeak', 
    8: 'Exfiltration', 
    9: 'Impact'
}

valid_ner = [
         'Known_Malware',
         'PRO',
         'PERSON',
         'filename',
         'uri',
         'md5',
         'ACTOR',
         'Known_Actor',
         'sha256',
         'URL',
         'sha1',
         'CVE',
]

def dedupe(article_extracts):
    seen_sent = dict()
    unique = []
    for triple in article_extracts:
        sent_ind = triple.sent_ind
        if sent_ind not in seen_sent:
            seen_sent[sent_ind] = list()
        
        dup = False
        for seen_triple in seen_sent[sent_ind]:
            if seen_triple == triple:
                if len(seen_triple.sub) >= len(triple.sub) and len(seen_triple.obj) >= len(triple.obj):
                    seen_sent[sent_ind].remove(seen_triple)
                    dup = False
                else:
                    #seen_triple.set_count(seen_triple.count+1)
                    dup = True
                break
                    
        if not dup:
            seen_sent[sent_ind].append(triple)
    
    for k, v in seen_sent.items():
        unique += v
        
    return unique
  
def fusion(trip):
    det_ner = ['filename', 'sha256', 'CVE', 'uri', 'md5', 'sha1']
    if trip.sub_ent not in det_ner:
        for single in malware:
            #if ' ' in single and single in trip.sub:
            if single in trip.sub:
                trip.sub = single
                trip.sub_ent = 'Known_Malware'
                trip.sub_ent_conf = 1
            elif single in trip.sub.split(' '):
                trip.sub = single
                trip.sub_ent = 'Known_Malware'
                trip.sub_ent_conf = 1
                
        for single in software:
           # if ' ' in single and single in trip.sub:
            if single in trip.sub:
                trip.sub_ent = 'PRO'
                trip.sub_ent_conf = 1
            elif single in trip.sub.split(' '):
                trip.sub_ent = 'PRO'
                trip.sub_ent_conf = 1
                
        for single in actor:
            #if ' ' in single and single in trip.sub:
            if single in trip.sub:
                trip.sub_ent = 'Known_Actor'
                trip.sub_ent_conf = 1
            elif single in trip.sub.split(' '):
                trip.sub_ent = 'Known_Actor'
                trip.sub_ent_conf = 1
    
    if trip.obj_ent not in det_ner:
        for single in malware:
            #if ' ' in single and single in trip.obj:
            if single in trip.obj:
                trip.obj = single
                trip.obj_ent = 'Known_Malware'
                trip.obj_ent_conf = 1
            elif single in trip.obj.split(' '):
                trip.obj = single
                trip.obj_ent = 'Known_Malware'
                trip.obj_ent_conf = 1
                
        for single in software:
            #if ' ' in single and single in trip.obj:
            if single in trip.obj:
                trip.obj_ent = 'PRO'
                trip.obj_ent_conf = 1
            elif single in trip.obj.split(' '):
                trip.obj_ent = 'PRO'
                trip.obj_ent_conf = 1
                
        for single in actor:
            #if ' ' in single and single in trip.obj:
            if single in trip.obj:
                trip.obj_ent = 'Known_Actor'
                trip.obj_ent_conf = 1
            elif single in trip.obj.split(' '):
                trip.obj_ent = 'Known_Actor'
                trip.obj_ent_conf = 1

def filter_ner(triples):
    filtered = []
    for triple in triples:
        if triple.sub_ent != '' and triple.sub_ent in valid_ner and triple.obj_ent != '' and triple.obj_ent in valid_ner:
            filtered.append(triple)
            
    return filtered

def filter_conf(triples, behav_thresh=0.5):
    filtered = []
    for triple in triples:        
        if triple.confidence < 0.8:
            continue
            
        if triple.behave_conf < behav_thresh:
            continue
        
        filtered.append(triple)
    return filtered

import numpy as np
import glob,pickle
import sys
from simpletransformers.classification import ClassificationModel,    ClassificationArgs,    MultiLabelClassificationModel # type: ignore

from simpletransformers.ner import NERModel
model = NERModel("roberta", "nermodel",use_cuda=True)
be_model=MultiLabelClassificationModel("roberta", 'behavior model',use_cuda=True)
ta_model=MultiLabelClassificationModel("roberta", 'tactic model',use_cuda=True)

def find_zeros(lst):
    zeros_count = 0
    zeros_indices = []
    for i in range(len(lst)):
        if lst[i] == 0:
            zeros_count += 1
        else:
            if zeros_count > 0:
                zeros_indices.append(i - zeros_count)
            zeros_count = 0
    if zeros_count > 0:
        zeros_indices.append(len(lst) - zeros_count)
    return zeros_indices

def nlp_process(dict_text_ann):
  anns = list(dict_text_ann.values())
  keys = list(dict_text_ann.keys())
  texts=[]
  tokenized_s=[]
  anns_noerror=[]
  for ann in anns:
    if ann != 'error':
      anns_noerror.append(ann)
      tokenized_s.append([[x.value for x in sent.token] for sent in ann.sentence])
      texts.append(ann.text)

  tokenized_s=[x[0] for x in tokenized_s]
  prediction, raw_output = model.predict(tokenized_s, split_on_space=False)

  extracted_listS=[]
  for i in range(len(anns_noerror)):
    ann=anns_noerror[i]
    ner=prediction[i]
    raw_out=raw_output[i]
    notext, ioc_dic = change_ioc(texts[i])
    extracted_list, entity_count, rel_count = extract_triples_with_ner(ann, ner, raw_out, ioc_dic, doc_id = 1)
    extracted_listS.append(extracted_list)

  text_list=texts

  be_predictions, be_raw_outputs = be_model.predict(text_list)
  ta_predictions, ta_raw_outputs = ta_model.predict(text_list)

  for i in range(len(extracted_listS)):
      if len(be_predictions[i])>0:
          for triple in extracted_listS[i]:
              triple.raw_sent=text_list[i]
              triple.behave_conf=be_raw_outputs[i][1]
              triple.tactic=find_zeros(ta_predictions[i])
              triple.sent_ind=i
              triple.tactic_conf=ta_predictions[i]

  for i in range(len(extracted_listS)):
    extracted_listS[i]=dedupe(extracted_listS[i])
  for i in range(len(extracted_listS)):
    for j in range(len(extracted_listS[i])):
      fusion(extracted_listS[i][j])
  return extracted_listS


In [ ]:
#Performs Entity Extraction on all files in the directory
import os
base_dir = '/home/user'
files = glob.glob(os.path.join(base_dir, 'noerror', '*.pkl'))
processed_files = glob.glob(os.path.join(base_dir, 'noerrorfinal', '*.pkl'))

processed_files_shortname = {os.path.basename(pf).replace('processed', '') for pf in processed_files}

# Use list comprehension to filter out processed files
files = [f for f in files if os.path.basename(f) not in processed_files_shortname]

import os
import pickle
import glob
from multiprocessing import Pool

for file in files:
    with open(file, 'rb') as f:
        try:
            my_dict = pickle.load(f)
            out = nlp_process(my_dict);
            out_file = os.path.join('/home/user', 'processed' + os.path.basename(file))
            with open(out_file, 'wb') as f_out:
                pickle.dump(out, f_out)
        except Exception as e:
            with open('error_afterpipeline.txt', 'a') as f_err:
                #print('error!')
                f_err.write(file + '\n')
                f_err.write(str(e) + '\n')

with open(file, 'rb') as f:
      my_dict=pickle.load(f)
      out=nlp_process(my_dict)

print('no output');